# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Code Lab 13: Wine** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez<br />
**Equipo**: JJAE<br />
**Integrantes**: Jose Cota, Jose Hurtado, Jose Mendoza, Andres Blanco

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("WineLab") \
    .master("spark://dc612074df78:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/09 17:09:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
from JJAE.spark_utils import SparkUtils

columns = [
    "Alcohol", "Malic_Acid", "Ash", "Ash_Alcanity", "Magnesium",
    "Total_Phenols", "Flavanoids", "Nonflavanoid_Phenols", "Proanthocyanins",
    "Color_Intensity", "Hue", "OD280", "Proline"
]

column_data_types = ["FloatType"] * len(columns)
columns_info = list(zip(columns, column_data_types))

schema = SparkUtils.generate_schema(columns_info)
print(schema)

StructType([StructField('Alcohol', FloatType(), True), StructField('Malic_Acid', FloatType(), True), StructField('Ash', FloatType(), True), StructField('Ash_Alcanity', FloatType(), True), StructField('Magnesium', FloatType(), True), StructField('Total_Phenols', FloatType(), True), StructField('Flavanoids', FloatType(), True), StructField('Nonflavanoid_Phenols', FloatType(), True), StructField('Proanthocyanins', FloatType(), True), StructField('Color_Intensity', FloatType(), True), StructField('Hue', FloatType(), True), StructField('OD280', FloatType(), True), StructField('Proline', FloatType(), True)])


In [16]:
wine_df = spark.read.schema(schema).option("header", "true").csv("/home/jovyan/notebooks/data/wine-clustering.csv")
wine_df.printSchema()
wine_df.show(5)

root
 |-- Alcohol: float (nullable = true)
 |-- Malic_Acid: float (nullable = true)
 |-- Ash: float (nullable = true)
 |-- Ash_Alcanity: float (nullable = true)
 |-- Magnesium: float (nullable = true)
 |-- Total_Phenols: float (nullable = true)
 |-- Flavanoids: float (nullable = true)
 |-- Nonflavanoid_Phenols: float (nullable = true)
 |-- Proanthocyanins: float (nullable = true)
 |-- Color_Intensity: float (nullable = true)
 |-- Hue: float (nullable = true)
 |-- OD280: float (nullable = true)
 |-- Proline: float (nullable = true)

+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+
|Alcohol|Malic_Acid| Ash|Ash_Alcanity|Magnesium|Total_Phenols|Flavanoids|Nonflavanoid_Phenols|Proanthocyanins|Color_Intensity| Hue|OD280|Proline|
+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+
|  14.23|      1.71|2.43

# ML vector


In [20]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

assembler = VectorAssembler(inputCols=columns, outputCol="Resultados")
assembled_df = assembler.transform(wine_df)

# Tomamos los valores
k_values = [2, 10, 15, 20]
models = []
predictions_list = []
silhouette_scores = []

evaluator = ClusteringEvaluator(featuresCol="Resultados", metricName="silhouette", distanceMeasure="squaredEuclidean")
print()

for k in k_values:
    kmeans = KMeans(featuresCol="Resultados", k=k, seed=1)
    model = kmeans.fit(assembled_df)
    models.append(model)
    
    predictions = model.transform(assembled_df)
    predictions_list.append(predictions)
    
    silhouette = evaluator.evaluate(predictions)
    silhouette_scores.append(silhouette)
    
    print(f"Silhouette score for k = {k}: {silhouette}")


Silhouette score for k = 2: 0.821360351333219
Silhouette score for k = 10: 0.6810238482372347
Silhouette score for k = 15: 0.6844672307118249
Silhouette score for k = 20: 0.6393344257369509


In [21]:
sc.stop()